In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [73]:
world_cup = pd.read_csv('./World Cup 2023 Dataset.csv')
results = pd.read_csv('./results.csv')

In [74]:
world_cup.head()

,Team,Group,Previous \nappearances,Previous \r\ntitles,Previous\r\n finals,Previous\r\n semifinals,Current \r rank
0,England,A,12,1,4,6,7
1,South Africa,A,7,0,0,5,2
2,Netherlands,A,5,0,0,0,10
3,Pakistan,A,12,1,2,6,5
4,New Zealand,A,12,0,2,9,4


In [75]:
results.head()

,date,Team_1,Team_2,Winner,Margin,Ground
0,"Jan 12, 2011",South Africa,India,South Africa,135 runs,Durban
1,"Jan 15, 2011",South Africa,India,India,1 run,Johannesburg
2,"Jan 16, 2011",Australia,England,Australia,6 wickets,Melbourne
3,"Jan 18, 2011",South Africa,India,India,2 wickets,Cape Town
4,"Jan 21, 2011",Australia,England,Australia,46 runs,Hobart


In [100]:
# Restricting to the teams in the 2023 World Cup.
worldcup_teams = ['England', 'South Africa', 'Netherlands', 'Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan', 'Australia', 'Bangladesh', 'India']

df_teams_1 = results[results['Team_1'].isin(worldcup_teams)]
df_teams_2 = results[results['Team_2'].isin(worldcup_teams)]

df_teams = pd.concat((df_teams_1, df_teams_2))
df_teams.drop_duplicates()

,date,Team_1,Team_2,Winner,Margin,Ground
0,"Jan 12, 2011",South Africa,India,South Africa,135 runs,Durban
1,"Jan 15, 2011",South Africa,India,India,1 run,Johannesburg
2,"Jan 16, 2011",Australia,England,Australia,6 wickets,Melbourne
3,"Jan 18, 2011",South Africa,India,India,2 wickets,Cape Town
4,"Jan 21, 2011",Australia,England,Australia,46 runs,Hobart
...,...,...,...,...,...,...
1524,"Jun 27, 2023",Scotland,Sri Lanka,Sri Lanka,82 runs,Bulawayo
1530,"Jul 2, 2023",Zimbabwe,Sri Lanka,Sri Lanka,9 wickets,Bulawayo
1543,"Jul 27, 2023",West Indies,India,India,5 wickets,Bridgetown
1544,"Jul 29, 2023",West Indies,India,West Indies,6 wickets,Bridgetown


In [101]:
#dropping columns that wll not affect match outcomes
df_teams_2010 = df_teams.drop(['date','Margin', 'Ground'], axis=1)
df_teams_2010

,Team_1,Team_2,Winner
0,South Africa,India,South Africa
1,South Africa,India,India
2,Australia,England,Australia
3,South Africa,India,India
4,Australia,England,Australia
...,...,...,...
1572,India,Australia,India
1573,Bangladesh,New Zealand,New Zealand
1575,India,Australia,India
1576,Bangladesh,New Zealand,New Zealand


In [102]:
#Building the model
#the prediction label: The winning_team column will show "1" Team 1 has won and "2" if the away team has won.

df_teams_2010 = df_teams_2010.reset_index(drop=True)
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team_1,'winning_team']=1
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team_2, 'winning_team']=2
df_teams_2010 = df_teams_2010.drop(['winning_team'], axis=1)

df_teams_2010

,Team_1,Team_2,Winner
0,South Africa,India,South Africa
1,South Africa,India,India
2,Australia,England,Australia
3,South Africa,India,India
4,Australia,England,Australia
...,...,...,...
2077,India,Australia,India
2078,Bangladesh,New Zealand,New Zealand
2079,India,Australia,India
2080,Bangladesh,New Zealand,New Zealand


In [79]:
#convert team-1 and team-2 from categorical variables to continous inputs 
# Get dummy variables
final = pd.get_dummies(df_teams_2010, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Separate X and y sets
X = final.drop(['Winner'], axis=1)
y = final["Winner"]


# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [97]:
final.head()

,Winner,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_Canada,Team_1_England,Team_1_Hong Kong,Team_1_India,Team_1_Ireland,Team_1_Kenya,...,Team_2_New Zealand,Team_2_Oman,Team_2_Pakistan,Team_2_Scotland,Team_2_South Africa,Team_2_Sri Lanka,Team_2_U.A.E.,Team_2_U.S.A.,Team_2_West Indies,Team_2_Zimbabwe
0,South Africa,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,India,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Australia,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,India,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Australia,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [81]:

rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)
rf.fit(X_train, y_train) 


score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.690
Test set accuracy:  0.613


In [82]:
#adding ICC rankings
#the team which is positioned higher on the ICC Ranking will be considered "favourite" for the match
#and therefore, will be positioned under the "Team_1" column

# Loading new datasets
ranking = pd.read_csv('./icc_rankings.csv') 
fixtures = pd.read_csv('./fixtures.csv')

# List for storing the group stage games
pred_set = []

In [83]:
# Create new columns with ranking position of each team
fixtures.insert(1, 'first_position', fixtures['Team_1'].map(ranking.set_index('Team')['Position']))
fixtures.insert(2, 'second_position', fixtures['Team_2'].map(ranking.set_index('Team')['Position']))

# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[:45, :]
fixtures.tail()

,Round Number,first_position,second_position,Date,Location,Team_1,Team_2,Group,Result
40,1,5.0,7.0,"Nov 9, 2023",Bengaluru,New Zealand,Sri Lanka,Group A,NaN
41,1,9.0,3.0,"Nov 10, 2023",Ahmedabad,Afghanistan,South Africa,Group A,NaN
42,1,2.0,8.0,"Nov 11, 2023",Pune,Australia,Bangladesh,Group A,NaN
43,1,6.0,4.0,"Nov 11, 2023",Eden Gardens,England,Pakistan,Group A,NaN
44,1,1.0,NaN,"Nov 12, 2023",Bengaluru,India,Netherlands,Group A,NaN


In [84]:
# Loop to add teams to new prediction dataset based on the ranking position of each team
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,Team_1,Team_2,winning_team
0,New Zealand,England,None
1,Pakistan,Netherlands,None
2,Bangladesh,Afghanistan,None
3,South Africa,Sri Lanka,None
4,India,Australia,None


In [85]:
# Get dummy variables and drop winning_team column
pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)
pred_set.head()

,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_Canada,Team_1_England,Team_1_Hong Kong,Team_1_India,Team_1_Ireland,Team_1_Kenya,Team_1_Nepal,...,Team_2_New Zealand,Team_2_Oman,Team_2_Pakistan,Team_2_Scotland,Team_2_South Africa,Team_2_Sri Lanka,Team_2_U.A.E.,Team_2_U.S.A.,Team_2_West Indies,Team_2_Zimbabwe
0,0,False,False,0,False,0,False,0,0,0,...,False,0,False,0,False,False,0,0,0,0
1,0,False,False,0,False,0,False,0,0,0,...,False,0,False,0,False,False,0,0,0,0
2,0,False,True,0,False,0,False,0,0,0,...,False,0,False,0,False,False,0,0,0,0
3,0,False,False,0,False,0,False,0,0,0,...,False,0,False,0,False,True,0,0,0,0
4,0,False,False,0,False,0,True,0,0,0,...,False,0,False,0,False,False,0,0,0,0


In [86]:
# making the points table for 2023 World Cup

points_table_df = pd.DataFrame(columns=['team', 'matches_played', 'won', 'lost', 'points'])
points_table_df.set_index('team', inplace=True)

for team in worldcup_teams:
    points_table_df.loc[team] = [0, 0, 0, 0]
points_table_df


,matches_played,won,lost,points
team,,,,
England,0,0,0,0
South Africa,0,0,0,0
Netherlands,0,0,0,0
Pakistan,0,0,0,0
New Zealand,0,0,0,0
Sri Lanka,0,0,0,0
Afghanistan,0,0,0,0
Australia,0,0,0,0
Bangladesh,0,0,0,0


In [87]:
#group matches 
predictions = rf.predict(pred_set)
for i in range(fixtures.shape[0]):
    team1 = backup_pred_set.iloc[i, 1]
    team2 = backup_pred_set.iloc[i, 0]

    points_table_df.loc[team1, 'matches_played'] += 1
    points_table_df.loc[team2, 'matches_played'] += 1
    
    print(team1 + " vs " + team2)
    if predictions[i] == 1:
        print("Winner: " + team1)
        points_table_df.loc[team1, 'won'] += 1
        points_table_df.loc[team2, 'lost'] += 1
        points_table_df.loc[team1, 'points'] += 2
    else:
        print("Winner: " + team2)
        points_table_df.loc[team2, 'won'] += 1
        points_table_df.loc[team1, 'lost'] += 1
        points_table_df.loc[team2, 'points'] += 2

    # print(points_table_df)
    print("")

England vs New Zealand
Winner: New Zealand

Netherlands vs Pakistan
Winner: Pakistan

Afghanistan vs Bangladesh
Winner: Bangladesh

Sri Lanka vs South Africa
Winner: South Africa

Australia vs India
Winner: India

Netherlands vs New Zealand
Winner: New Zealand

Bangladesh vs England
Winner: England

Sri Lanka vs Pakistan
Winner: Pakistan

Afghanistan vs India
Winner: India

South Africa vs Australia
Winner: Australia

Bangladesh vs New Zealand
Winner: New Zealand

Pakistan vs India
Winner: India

Afghanistan vs England
Winner: England

Sri Lanka vs Australia
Winner: Australia

Netherlands vs South Africa
Winner: South Africa

Afghanistan vs New Zealand
Winner: New Zealand

Bangladesh vs India
Winner: India

Pakistan vs Australia
Winner: Australia

Netherlands vs Sri Lanka
Winner: Sri Lanka

England vs South Africa
Winner: South Africa

New Zealand vs India
Winner: India

Afghanistan vs Pakistan
Winner: Pakistan

Bangladesh vs South Africa
Winner: South Africa

Australia vs Netherlands


In [88]:
points_table_df = points_table_df.sort_values(by='points', ascending=False)
print(points_table_df)

              matches_played  won  lost  points
team                                           
India                      9    8     1      16
South Africa               9    7     2      14
Australia                  9    7     2      14
Pakistan                   9    6     3      12
Netherlands                9    5     4      10
New Zealand                9    5     4      10
England                    9    3     6       6
Sri Lanka                  9    3     6       6
Bangladesh                 9    1     8       2
Afghanistan                9    0     9       0


In [89]:
# Assuming you have already sorted the DataFrame and obtained the top 4 teams
sorted_points_table = points_table_df.sort_values(by='points', ascending=False)
top_4_teams = sorted_points_table.head(4)

# Accessing the names of the top 4 teams
top_4_team_names = top_4_teams.index.tolist()

# Displaying the result
print(top_4_team_names)


['India', 'South Africa', 'Australia', 'Pakistan']


In [90]:
# List of tuples before 
semi = [(top_4_team_names[0], top_4_team_names[3]),
            (top_4_team_names[1], top_4_team_names[2])]

In [91]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Position'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Position'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'Team_1': matches[j][0], 'Team_2': matches[j][1]})
        else:
            dict1.update({'Team_1': matches[j][1], 'Team_2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    
    if len(pred_set) == 2 :
        print("---------SEMI FINALS-----------\n")
    else :
        print("----------FINALS---------\n")

    # Predict!
    predictions = logreg.predict(pred_set)
    winners = []
    for i in range(len(pred_set)):
        team1 = backup_pred_set.iloc[i, 1]
        team2 = backup_pred_set.iloc[i, 0]

        print(team1 + " vs " + team2)
        if predictions[i] == 1:
            print("Winner: " + team1)
            winners.append(team1)
        else:
            print("Winner: " + team2)
            winners.append(team2)
        print("")

    if len(winners) == 2 :
        return [(winners[0], winners[1])]
    else :
        return winners[0]

In [92]:
# Results of semi finals
finals = clean_and_predict(semi, ranking, final, rf)

---------SEMI FINALS-----------

Pakistan vs India
Winner: India

South Africa vs Australia
Winner: Australia



In [93]:
print(finals)

[('India', 'Australia')]


In [94]:
# Result of the finals

wc_winner = clean_and_predict(finals, ranking, final, rf)

print(f"{wc_winner} will win the 2023 Cricket World Cup")

----------FINALS---------

Australia vs India
Winner: India

India will win the 2023 Cricket World Cup


In [95]:
### Create a Pickle file using serialization
import pickle
pickle_out = open("classifier.pkl","wb")
pickle.dump(wc_winner, pickle_out)
#Serialization
pickle_out.close()